In [9]:
import pandas as pd
from tqdm import tqdm
import gc
import pickle
import random
from datetime import datetime
from operator import itemgetter
import numpy as np
import pandas as pd
import warnings
from collections import defaultdict
import collections
warnings.filterwarnings('ignore')

In [6]:
path = 'C:/ZhangLI/Codes/DataSet/新闻推荐比赛数据/'
train = path + 'articles.csv'
train_click = path + 'train_click_log.csv'

In [7]:
# 使用历史浏览 点击文章的数据信息预测用户未来的点击行为，即用户最后一次点击新闻的文章
# 问题转换 30万用户 200w点击 36w篇文章
train_df = pd.read_csv(train)
train_click_df = pd.read_csv(train_click)

In [13]:
train_click_df.memory_usage().sum()/1024**2

76.39788055419922

In [14]:
def reduce_memory(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_types = df[col].dtypes
        if col_types in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_types)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem, 
                                                                                                    100*(start_mem-end_mem)/start_mem, 
                                                                                                    (time.time()-starttime)/60))
    return df

In [17]:
# 定义
user_recall_items_dict = collections.defaultdict(dict)
user_recall_items_dict

defaultdict(dict, {})

In [24]:
import copy
all_click_df = copy.deepcopy(train_click_df)
c = all_click_df.sort_values('click_timestamp')
# 这应该是个键值对list(zip(c['click_article_id'], c['click_timestamp']))
c.groupby('user_id')['click_article_id', 'click_timestamp']

[(157770, 1507029532200),
 (272143, 1507029551778),
 (70594, 1507029553199),
 (272143, 1507029564434),
 (160417, 1507029570190),
 (70755, 1507029570910),
 (5408, 1507029571478),
 (156624, 1507029580386),
 (156624, 1507029582008),
 (272143, 1507029583199),
 (156624, 1507029584077),
 (160417, 1507029586033),
 (156624, 1507029587080),
 (283123, 1507029589145),
 (158229, 1507029591819),
 (169111, 1507029596200),
 (5408, 1507029596628),
 (299697, 1507029600494),
 (67542, 1507029600910),
 (156624, 1507029601180),
 (50823, 1507029601478),
 (149599, 1507029611062),
 (160974, 1507029612008),
 (207122, 1507029616033),
 (286336, 1507029619145),
 (160417, 1507029620497),
 (160417, 1507029621819),
 (198659, 1507029626200),
 (272143, 1507029626628),
 (156624, 1507029627452),
 (16129, 1507029630494),
 (156447, 1507029631180),
 (160417, 1507029634489),
 (156624, 1507029634566),
 (156560, 1507029637160),
 (50823, 1507029639296),
 (156624, 1507029647927),
 (272143, 1507029652091),
 (156624, 150702965349

In [25]:
c

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
3,199998,157770,1507029532200,4,1,17,1,25,5
45,199987,272143,1507029551778,4,1,17,1,24,5
52,199984,70594,1507029553199,4,3,2,1,5,7
32,199990,272143,1507029564434,4,1,17,1,7,5
0,199999,160417,1507029570190,4,1,17,1,13,1
...,...,...,...,...,...,...,...,...,...
705656,121158,224148,1509798422502,4,1,17,1,13,2
925954,70254,207672,1510093882860,4,1,17,1,20,2
925955,70254,96333,1510093912860,4,1,17,1,20,2
1091177,2465,203538,1510603424886,4,1,17,1,2,2


In [ ]:
# 根据点击时间获取用户的点击文章序列 {user1: [(item1, time1), (item2, item2)..]..}
def get_user_item_time(click_df):
    